# Create NIRCam TSO Simulated Data

This is a bare-bones notebook designed to show how to run Mirage to create TSO data. It patches together the contents of several scripts written during the development of TSO mode in Mirage.

In [ ]:
from astropy.io import fits, ascii
from astropy.table import Table
import batman
import h5py
import numpy as np
import matplotlib.pyplot as plt

from mirage.catalogs.hdf5_catalog import save_tso
from mirage.grism_tso_simulator import GrismTSO
from mirage.imaging_simulator import ImgSim
from mirage.seed_image.catalog_seed_image import Catalog_seed

## Grism TSO

1. Get a stellar spectrum
2. Create a transmission spectrum
3. Create source catalog(s) for Mirage
4. Create an input yaml file for Mirage
5. Run Mirage

### Get a stellar spectrum

This is the spectrum of the unocculted star associated with the TSO object. Example below is a simple flat continuum. Another option for getting a spectrum would be to use pysynphot.

In [ ]:
target_1_wavelength = np.arange(1.0, 5.5, 0.1)
target_1_flux = np.repeat(1e-16, len(target_1_wavelength))
wavelengths = [target_1_wavelength]
fluxes = [target_1_flux]

In [ ]:
wavelength_units = 'microns'
flux_units = 'flam'

In [ ]:
sed_file = 'test_grism_tso_sed_file.hdf5'

In [ ]:
with h5py.File(sed_file, "w") as file_obj:
    for i in range(len(fluxes)):
        dset = file_obj.create_dataset(str(i+1), data=[wavelengths[i], fluxes[i]], dtype='f',
                                       compression="gzip", compression_opts=9)
        dset.attrs[u'wavelength_units'] = wavelength_units
        dset.attrs[u'flux_units'] = flux_units

### Create source catalog(s) for Mirage

Mirage accepts a series of ascii source catalogs. In this case, the options that can be used are:

1. TSO grism catalog
2. Point source catalog
3. Galaxy catalog
4. Extended source catalog

The point source, galaxy, and extended source catalogs contain "background" sources. That is, sources other than the primary TSO target. All catalogs are input to Mirage as entries within the input yaml file.

See the notebook on catalog creation in the examples directory of the Mirage repository for examples of how to create source catalogs. Catalogs may also be created manually.

In [ ]:
tso_catalog = 'tso_grism_source.cat'
ptsrc_catalog = 'ptsrcs.cat'

### Create a transmission spectrum

The transmission spectrum is the wavelength-dependent effective radius of the planet, in units of the stellar radius. The transmission spectrum must be in an ASCII file.

In [ ]:
waves = np.linspace(0.9, 5.5, 1000)  # microns
trans = np.repeat(1.0, 1000)  # R_planet / R_star

In [ ]:
tab = Table()
tab['Wavelength'] = waves
tab['Transmission'] = trans
tab.write('transmission_spectrum.txt', format='ascii')

### Create an input yaml file for Mirage

This file contains all the inputs needed by Mirage. For most Mirage simulations, there is a tool to create these files from an APT file. However, TSO mode has not yet been added to this tool, so yaml files will have to be produced manually for the time being.

In [ ]:
paramfile = 'jw_grism_tso_test_nrcb5.yaml'

### Run Mirage

Gather together the inputs and call Mirage's `tso_simulator`

In [ ]:
m = GrismTSO(paramfile, SED_file=sed_file, SED_normalizing_catalog_column=None,
                 final_SED_file=None, save_dispersed_seed=True, source_stamps_file=None,
                 extrapolate_SED=True, override_dark=None, disp_seed_filename=None, orders=["+1", "+2"],
                 create_continuum_seds=True)
m.create()

## Imaging TSO 

1. Create lightcurve file
2. Create source catalog(s) for Mirage
3. Create an input yaml file for Mirage
4. Run Mirage

### Create lightcurve file

Use batman to create a lightcurve as save the results to an hdf5 file, which can then be input to Mirage

In [ ]:
catalog_name = 'example_lightcurve.hdf5'

In [ ]:
# Batman parameters - these have been copied from the example in the Batman documentation

params = batman.TransitParams()       # object to store transit parameters
params.t0 = 0.                        # time of inferior conjunction
params.per = 1.                       # orbital period
params.rp = 0.8                       # planet radius (in units of stellar radii)
params.a = 10.                        # semi-major axis (in units of stellar radii)
params.inc = 87.                      # orbital inclination (in degrees)
params.ecc = 0.                       # eccentricity
params.w = 90.                        # longitude of periastron (in degrees)
params.limb_dark = "nonlinear"        # limb darkening model
params.u = [0.5, 0.1, 0.1, -0.1]      # limb darkening coefficients [u1, u2, u3, u4]

In [ ]:
times = np.linspace(-0.025, 0.025, 1000)  # times at which to calculate light curve
m = batman.TransitModel(params, times)    # initializes model
flux = m.light_curve(params)              # calculates light curve

In [ ]:
# Place the lightcurve into a dictionary to prepare for saving. The keys are object
# numbers corresponding to objects in the Mirage input catalogs.
example_times = np.linspace(0, 110., 1000)

contents = {}
contents['7'] = {'times': example_times,
                 'fluxes': flux}
contents['8'] = {'times': example_times,
                 'fluxes': flux}

In [ ]:
# Save
save_tso(contents, catalog_name, time_unit='second')

In [ ]:
# Plot the lightcurve
f, a = plt.subplots()
a.scatter(example_times, flux, color='red', marker='v')
plt.show()

### Create source catalog(s) for Mirage

In this case, rather than the `tso_grism_catalog` in the yaml file, the user must supply a `tso_imaging_catalog`. This catalog will contain the list of sources whose flux will be varying with time. As in the grism TSO case, objects listed in the other types of catalogs will be added to the simulation in order to create a more realistic scene.

See the notebook on catalog creation in the examples directory of the Mirage repository for examples of how to create source catalogs. Catalogs may also be created manually.

In [ ]:
tso_imaging_catalog = 'example_tso_sources.cat'

### Create an input yaml file for Mirage

This file contains all the inputs needed by Mirage. For most Mirage simulations, there is a tool to create these files from an APT file. However, TSO mode has not yet been added to this tool, so yaml files will have to be produced manually for the time being.

In [ ]:
imaging_yaml = 'jw_add_tso_source.yaml'

### Run Mirage

Option 1: create just the noiseless seed image. This is helpful when testing inputs, as skipping the later steps of Mirage saves significant time

In [ ]:
c = Catalog_seed()
c.paramfile = imaging_yaml
c.make_seed()

Option 2: Run all steps of Mirage, in order to create the final simulated data

In [ ]:
c = ImgSim()
c.paramfile = imaging_yaml
c.create()